# Revolve Tatbletennis ELO ladder
# Author: Harald Lønsethagen
# Started: 28.08.2017

## Importing Libraries

In [76]:
import pandas as pd
import numpy as np
import matplotlib as plot

## Defining usefull functions

In [77]:
def expected(A, B):
    """
    Calculate expected score of A in a match against B
    :param A: Elo rating for player A
    :param B: Elo rating for player B
    """
    return 1 / (1 + 10 ** ((B - A) / 400))


def elo(old, exp, score, k=32):
    """
    Calculate the new Elo rating for a player
    :param old: The previous Elo rating
    :param exp: The expected score for this match
    :param score: The actual score for this match
    :param k: The k-factor for Elo (default: 32)
    """
    return old + k * (score - exp)

## Importing all played matches

In [78]:
matches_df = pd.read_csv("matches.csv")
print("Matches: " + str(matches_df.shape[0]))
matches_df

Matches: 186


,Player 1,Player 2,Winner,timestamp
0,Paul Frivold,Robert Tamang,Robert Tamang,2017-09-08
1,Paul Frivold,Bo Woelfert,Bo Woelfert,2017-09-08
2,Mons,Ole Andreas,Mons,2017-09-09
3,Mons,Eirik B.,Mons,2017-09-09
4,Mons,Eirik B.,Mons,2017-09-09
5,Paul Frivold,Dino,Paul Frivold,2017-09-09
6,Paul Frivold,Mons,Mons,2017-09-09
7,Bo Woelfert,Dino,Bo Woelfert,2017-09-09
8,Jakob,Mons,Mons,2017-09-10
9,Mons,Yohann,Mons,2017-09-10


## Importing all players 

In [79]:
players_df = pd.read_csv("players.csv")
print("Players: " + str(players_df.shape[0]))
players_df

Players: 50


,first_name,last_name,short_name,joined_date,id
0,Amund,NaN,rt,17-09-08,1
1,Anders,NaN,mbe,17-09-08,2
2,Andreas B.,NaN,NaN,17-09-07,3
3,Andreas H.,NaN,NaN,17-09-06,4
4,Bo Woelfert,NaN,NaN,17-09-06,5
5,Didrik,NaN,NaN,17-09-04,6
6,Dino,NaN,NaN,17-09-03,7
7,Eirik B.,NaN,NaN,17-09-02,8
8,Even,NaN,NaN,17-09-02,9
9,Fabian S. Rønningen,NaN,NaN,17-09-01,10


## ELO-ladder Dataframe containing the ELO-score to each player

In [80]:
elo_df = players_df
elo_df['elo'] = 1000
print("Size of ELO-df: " + str(elo_df.shape[0]))
elo_df.head()

Size of ELO-df: 50


,first_name,last_name,short_name,joined_date,id,elo
0,Amund,NaN,rt,17-09-08,1,1000
1,Anders,NaN,mbe,17-09-08,2,1000
2,Andreas B.,NaN,NaN,17-09-07,3,1000
3,Andreas H.,NaN,NaN,17-09-06,4,1000
4,Bo Woelfert,NaN,NaN,17-09-06,5,1000


# Function for calculating ELO

In [81]:
matches_df.iloc(1)[0]

Player 1      Paul Frivold
Player 2     Robert Tamang
Winner       Robert Tamang
timestamp       2017-09-08
Name: 0, dtype: object

In [82]:
length = matches_df.shape[0]
print("Number of matches: " + str(length) + "\n")
for i in range(length):
    match = matches_df.iloc[i]
    print(str(i) +": " + str(match['Player 1']) + " - " + str(match['Player 2']))
    player_1 = match["Player 1"]
    player_2 = match["Player 2"]
    elo_rating_a = elo_df[elo_df['first_name'] == player_1]['elo']
    elo_rating_b = elo_df[elo_df['first_name'] == player_2]['elo']

    if(elo_rating_a.shape[0] == 0):
        print("========================================================================")
        print("Player named '" + str(player_1) + "' is NOT FOUND in the player database. ")
        print("========================================================================")
        break
    if(elo_rating_b.shape[0] == 0):
        print("========================================================================")
        print("Player named '" + str(player_2) + "' is NOT FOUND in the player database. ")
        print("========================================================================")
        break
    elo_rating_a = int(elo_rating_a.values[0])
    elo_rating_b = int(elo_rating_b.values[0])
    
    
    if(match['Winner'] == player_1): 
        winner_a = 1
        winner_b = 0
    else:
        winner_a = 0
        winner_b = 1
    expected_a  = expected(elo_rating_a, elo_rating_b)
    expected_b  = expected(elo_rating_b, elo_rating_a)
    
    new_elo_a = elo(elo_rating_a, expected_a, winner_a, k = 50)
    new_elo_b = elo(elo_rating_b, expected_b, winner_b, k = 50)
    #print("New elo a: " + str(new_elo_a))
    #print("New elo b: " + str(new_elo_b))
    print("Player 1: '" + player_1 + "'. Old elo: " + str(elo_rating_a) + "'. New elo: " + str(new_elo_a) + ". exp: " + str(expected_a) + " " )
    print("Player 1: '" + player_2 + "'. Old elo: " + str(elo_rating_b) + "'. New elo: " + str(new_elo_b) + ". exp: " + str(expected_b) + " " )
    elo_df = elo_df.set_index('first_name')
    elo_df = elo_df.set_value(player_1, 'elo', new_elo_a + 1)
    elo_df = elo_df.set_value(player_2, 'elo', new_elo_b + 1)
    elo_df = elo_df.reset_index()
    
elo_df.head()

Number of matches: 186

0: Paul Frivold - Robert Tamang
Player 1: 'Paul Frivold'. Old elo: 1000'. New elo: 975.0. exp: 0.5 
Player 1: 'Robert Tamang'. Old elo: 1000'. New elo: 1025.0. exp: 0.5 
1: Paul Frivold - Bo Woelfert
Player 1: 'Paul Frivold'. Old elo: 976'. New elo: 952.7241972363415. exp: 0.46551605527316847 
Player 1: 'Bo Woelfert'. Old elo: 1000'. New elo: 1023.2758027636585. exp: 0.5344839447268315 
2: Mons - Ole Andreas
Player 1: 'Mons'. Old elo: 1000'. New elo: 1025.0. exp: 0.5 
Player 1: 'Ole Andreas'. Old elo: 1000'. New elo: 975.0. exp: 0.5 
3: Mons - Eirik B.
Player 1: 'Mons'. Old elo: 1026'. New elo: 1049.1326341416755. exp: 0.5373473171664928 
Player 1: 'Eirik B.'. Old elo: 1000'. New elo: 976.8673658583247. exp: 0.4626526828335072 
4: Mons - Eirik B.
Player 1: 'Mons'. Old elo: 1050'. New elo: 1069.8231840736912. exp: 0.6035363185261781 
Player 1: 'Eirik B.'. Old elo: 977'. New elo: 957.1768159263089. exp: 0.39646368147382194 
5: Paul Frivold - Dino
Player 1: 'Paul F

Player 1: 'Amund'. Old elo: 986'. New elo: 1005.7543548996022. exp: 0.6049129020079569 
Player 1: 'Dino'. Old elo: 912'. New elo: 892.2456451003978. exp: 0.3950870979920432 
66: Amund - Bo Woelfert
Player 1: 'Amund'. Old elo: 1006'. New elo: 987.6703850035584. exp: 0.3665922999288324 
Player 1: 'Bo Woelfert'. Old elo: 1101'. New elo: 1119.3296149964417. exp: 0.6334077000711676 
67: Mons - Cornelia
Player 1: 'Mons'. Old elo: 1126'. New elo: 1143.9967500098558. exp: 0.6400649998028851 
Player 1: 'Cornelia'. Old elo: 1026'. New elo: 1008.0032499901442. exp: 0.35993500019711494 
68: Odin A.S. - Anders
Player 1: 'Odin A.S.'. Old elo: 918'. New elo: 901.752010052015. exp: 0.3249597989597013 
Player 1: 'Anders'. Old elo: 1045'. New elo: 1061.2479899479852. exp: 0.6750402010402987 
69: Mathias - Harald Lønsethagen
Player 1: 'Mathias'. Old elo: 953'. New elo: 931.3614422171476. exp: 0.4327711556570478 
Player 1: 'Harald Lønsethagen'. Old elo: 1000'. New elo: 1021.6385577828524. exp: 0.567228844

142: Sondre N.A. - Lars Erik
Player 1: 'Sondre N.A.'. Old elo: 984'. New elo: 960.2940465445585. exp: 0.4741190691088309 
Player 1: 'Lars Erik'. Old elo: 1002'. New elo: 1025.7059534554417. exp: 0.525880930891169 
143: Ole Andreas - Kristoffer H
Player 1: 'Ole Andreas'. Old elo: 1004'. New elo: 1031.0104114161873. exp: 0.45979177167625435 
Player 1: 'Kristoffer H'. Old elo: 1032'. New elo: 1004.9895885838127. exp: 0.5402082283237456 
144: Aksel - Lars Erik
Player 1: 'Aksel'. Old elo: 976'. New elo: 954.5731558704191. exp: 0.4285368825916186 
Player 1: 'Lars Erik'. Old elo: 1026'. New elo: 1047.426844129581. exp: 0.5714631174083814 
145: Didrik - Yohann
Player 1: 'Didrik'. Old elo: 960'. New elo: 942.3991506625152. exp: 0.3520169867496943 
Player 1: 'Yohann'. Old elo: 1066'. New elo: 1083.6008493374848. exp: 0.6479830132503057 
146: Robert Tamang - Mats
Player 1: 'Robert Tamang'. Old elo: 1179'. New elo: 1190.8557509847374. exp: 0.7628849803052542 
Player 1: 'Mats'. Old elo: 976'. New e

,first_name,last_name,short_name,joined_date,id,elo
0,Amund,NaN,rt,17-09-08,1,980
1,Anders,NaN,mbe,17-09-08,2,1045
2,Andreas B.,NaN,NaN,17-09-07,3,1136
3,Andreas H.,NaN,NaN,17-09-06,4,1026
4,Bo Woelfert,NaN,NaN,17-09-06,5,1094


In [83]:
elo_df

,first_name,last_name,short_name,joined_date,id,elo
0,Amund,NaN,rt,17-09-08,1,980
1,Anders,NaN,mbe,17-09-08,2,1045
2,Andreas B.,NaN,NaN,17-09-07,3,1136
3,Andreas H.,NaN,NaN,17-09-06,4,1026
4,Bo Woelfert,NaN,NaN,17-09-06,5,1094
5,Didrik,NaN,NaN,17-09-04,6,969
6,Dino,NaN,NaN,17-09-03,7,955
7,Eirik B.,NaN,NaN,17-09-02,8,958
8,Even,NaN,NaN,17-09-02,9,967
9,Fabian S. Rønningen,NaN,NaN,17-09-01,10,1095


In [84]:
temp_df = elo_df.sort_values('elo', ascending=False)[['first_name','elo']]
temp_df

,first_name,elo
24,Robert Tamang,1228
19,Mons,1183
2,Andreas B.,1136
35,Harald Lønsethagen,1115
9,Fabian S. Rønningen,1095
4,Bo Woelfert,1094
28,Thomas H,1054
29,Yohann,1052
46,Hans,1052
43,Lars Erik,1048


# Save ELO-leaderboard to a csv file

In [41]:
temp_df.to_csv("elo_leaderboard.csv")

# Notes

In [54]:
elo_df = elo_df.reset_index()

In [55]:
elo_df[elo_df['first_name'] == "Harald"]['elo']

0    1001
Name: elo, dtype: int64

In [56]:
elo_df = elo_df.set_index('first_name')
elo_df.set_value('Harald', 'elo', 3902)
elo_df.reset_index()

,first_name,index,last_name,short_name,joined_date,id,elo
0,Harald,0,Lønsethagen,hlø,17-09-05,1,3902
1,Mathias,1,Becksæther,mbe,17-09-05,2,1001


In [57]:
elo_df = elo_df.reset_index()

In [58]:
elo_df = elo_df.set_index('first_name')
elo_df.set_value('Harald', 'elo', 900)

,index,last_name,short_name,joined_date,id,elo
first_name,,,,,,
Harald,0,Lønsethagen,hlø,17-09-05,1,900
Mathias,1,Becksæther,mbe,17-09-05,2,1001


In [59]:
exp  = expected(1613, 1609)
exp += expected(1613, 1477)
exp

1.1920564660944573

In [23]:
elo(1613, exp, 2, k=32)  # 1601

1638.8541930849774

In [24]:
a = expected(1613, 1609)
temp_elo = elo(1613, a, 1, k=32)
print("temp_elo: " + str(temp_elo))
a = expected(temp_elo, 1477)
temp_elo = elo(temp_elo, a , 1, k=32)
print("temp_elo after second match: " + str(temp_elo))

temp_elo: 1628.8158013308434
temp_elo after second match: 1638.2378451931402


In [ ]:
# Defining new elo_table with 

# ===============
#          Notes 
# ===============

## Defining usefull DataFrames with Pandas
- All played matches

In [9]:
#matches_df = pd.dataFrame()
item_1 = pd.Series({'timestamp': '00-00-00',
                        'Player 1': 'Harald',
                        'Player 2': 'Petter',
                        'Winner': 'Harald'})
matches_df = pd.DataFrame([item_1])
matches_df.to_csv("matches.csv")
matches_df

,Player 1,Player 2,Winner,timestamp
0,Harald,Petter,Harald,00-00-00
